In [8]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain_community.callbacks import get_openai_callback
import os, dotenv

dotenv.load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

MODEL = "gpt-4-1106-preview"

In [14]:
synopsis = """
주요 캐릭터:

1. 탐정 (플레이어): 유명한 탐정으로, 특이한 사건을 해결하는 데 능숙함.
2. 촌장: 마을의 촌장으로, 마을을 이끌고 있는 지혜로운 촌장.
3. 마을 주민들: 각기 다른 특성과 배경을 가진 주민들.

스토리 개요:

- 시작: 탐정인 플레이어는 마을의 촌장으로부터 마을에서 발생한 충격적인 살인 사건에 대한 의뢰를 받음. 평화로웠던 마을에서 처음으로 발생한 이 사건은 모두를 불안하게 만듬.
- 게임 진행: 탐정은 낮 동안 마을을 돌아다니며 주민들과 대화를 통해 힌트나 거짓말 을 판별하여 범인을 추리해야함.
- 밤 시간: 밤이 되면 범인은 또 다른 주민을 살해함. 이로 인해 다음 날의 조사는 더 어려워지며, 새로운 증거와 정보가 등장.
- 결말: 탐정은 모든 대화를 통해 범인을 추리해야 함. 범인을 정확히 지목하면 게임에서 승리, 실패 시 게임 오버.
"""

chain_suffix = """
{input}
"""

In [23]:
my_prompt = """
주요 캐릭터:

1. 탐정 (플레이어): 유명한 탐정으로, 특이한 사건을 해결하는 데 능숙함.
2. 촌장: 마을의 촌장으로, 마을을 이끌고 있는 지혜로운 촌장.
3. 마을 주민들: 각기 다른 특성과 배경을 가진 주민들.

스토리 개요:

- 시작: 탐정인 플레이어는 마을의 촌장으로부터 마을에서 발생한 충격적인 살인 사건에 대한 의뢰를 받음. 평화로웠던 마을에서 처음으로 발생한 이 사건은 모두를 불안하게 만 듬.
- 게임 진행: 탐정은 낮 동안 마을을 돌아다니며 주민들과 대화를 통해 힌트나 거짓말 을 판별하여 범인을 추리해야함.
- 밤 시간: 밤이 되면 범인은 또 다른 주민을 살해함. 이로 인해 다음 날의 조사는 더 어려워지며, 새로운 증거와 정보가 등장.
- 결말: 탐정은 모든 대화를 통해 범인을 추리해야 함. 범인을 정확히 지목하면 게임에서 승리, 실패 시 게임 오버.

1. information을 참고하여 character를 연기하여 user의 chatContent에 대답해야 함    
2. 답변의 길이는 2~3 문장 안으로 제한함.
3. PreviousChatContents가 주어진다면 해당 대화에 맞는 답변을 생성해야 함.
4. 지난 밤에 무엇을 했는지 물어본다면 alibi 항목을 참고
5. 답변을 만들 때 personalityDescription, featureDescription을 참고해야 함
"""

In [24]:
inputs = """
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"chatContent": {"title": "Chatcontent", "type": "string"}}, "required": ["chatContent"]}
```
"""

In [25]:
string = """
information: 
    user: 형구
    character:
        name: 레오
        personalityDescription: 용기 있게 행동함
        featureDescription: 노래를 부르듯이 대답함
        alibi: 지난 밤 마을 잔치에 참여함
    chatContent: 내 이름이 뭔지 알아?
    previousStory: 
    previousChatContents: 
        type: user
        name: 형구
        content: 넌 누구야?

        type: character
        name: 레오
        content: 안녕하신가, 나는 레오라고 하네. 용기 있게 행동하는 것을 좋아하고, 대답할 땐 마치 노래를 부르듯이 말하네.
"""

In [26]:
type_json_pre = """
{
  "information": {
    "user": "형구",
    "character": {
      "name": "레오",
      "personalityDescription": "용기 있게 행동함",
      "featureDescription": "노래를 부르듯이 대답함",
      "alibi": "지난 밤 마을 잔치에 참여함"
    },
    "chatContent": "내 이름이 뭔지 알아?",
    "previousStory": "",
    "previousChatContents": [
      {
        "type": "user",
        "name": "형구",
        "content": "넌 누구야?"
      },
      {
        "type": "character",
        "name": "레오",
        "content": "안녕하신가, 나는 레오라고 하네. 용기 있게 행동하는 것을 좋아하고, 대답할 땐 마치 노래를 부르듯이 말하네."
      }
    ]
  }
}
"""

In [27]:
type_json = """
{'information': {'user': '형구', 'character': {'name': '레오', 'personalityDescription': '용기 있게 행동함', 'featureDescription': '노래를 부르듯이 대답함', 'alibi': '지난 밤  마을 잔치에 참여함'}, 'chatContent': '내 이름이 뭔지 알아?', 'previousStory': '', 'previousChatContents': [{'type': 'user', 'name': '형구', 'content': '넌 누구야?'}, {'type': 'character', 'name': '레오', 'content': '안녕하신가, 나는 레오라고 하네. 용기 있게 행동하는 것을 좋아하고, 대답할 땐 마치 노래를 부르듯이 말하네.'}]}}
"""

In [28]:
type_pydantic = """
information=ConversationWithUserGeneration(user='형구', character=CharacterInfo(name='레오', personalityDescription='용기 있게 행동함', featureDescription='노래를 부르듯이 대답함', alibi='지난 밤 마을 잔치에 참여함'), chatContent='내 이름이 뭔지 알아?', previousStory='', previousChatContents=[PreviousChatContents(type='user', name='형구', content='넌 누구야?'), PreviousChatContents(type='character', name='레오', content='안녕하신가, 나 는 레오라고 하네. 용기 있게 행동하는 것을 좋아하고, 대답할 땐 마치 노래를 부르듯이 말하 네.')])
"""

In [42]:
# template = my_prompt + chain_suffix
template = chain_suffix
prompt = PromptTemplate(template=template, input_variables=["input"])

In [46]:
def generate(inputs):
    llm = ChatOpenAI(model=MODEL, openai_api_key=OPENAI_API_KEY)
    chain = LLMChain(
            prompt=prompt,
            llm=llm,
            verbose=True,
        )

    with get_openai_callback() as cb:
        response = chain.predict(input=inputs)

        tokens = {
            "totalTokens": cb.total_tokens, 
            "promptTokens": cb.prompt_tokens, 
            "completionTokens": cb.completion_tokens,
        }

    return response, tokens

In [32]:
response, tokens = generate(inputs + string)
print(response)
print(tokens)



> Entering new LLMChain chain...
Prompt after formatting:

주요 캐릭터:

1. 탐정 (플레이어): 유명한 탐정으로, 특이한 사건을 해결하는 데 능숙함.
2. 촌장: 마을의 촌장으로, 마을을 이끌고 있는 지혜로운 촌장.
3. 마을 주민들: 각기 다른 특성과 배경을 가진 주민들.

스토리 개요:

- 시작: 탐정인 플레이어는 마을의 촌장으로부터 마을에서 발생한 충격적인 살인 사건에 대한 의뢰를 받음. 평화로웠던 마을에서 처음으로 발생한 이 사건은 모두를 불안하게 만 듬.
- 게임 진행: 탐정은 낮 동안 마을을 돌아다니며 주민들과 대화를 통해 힌트나 거짓말 을 판별하여 범인을 추리해야함.
- 밤 시간: 밤이 되면 범인은 또 다른 주민을 살해함. 이로 인해 다음 날의 조사는 더 어려워지며, 새로운 증거와 정보가 등장.
- 결말: 탐정은 모든 대화를 통해 범인을 추리해야 함. 범인을 정확히 지목하면 게임에서 승리, 실패 시 게임 오버.

1. information을 참고하여 character를 연기하여 user의 chatContent에 대답해야 함    
2. 답변의 길이는 2~3 문장 안으로 제한함.
3. PreviousChatContents가 주어진다면 해당 대화에 맞는 답변을 생성해야 함.
4. 지난 밤에 무엇을 했는지 물어본다면 alibi 항목을 참고
5. 답변을 만들 때 personalityDescription, featureDescription을 참고해야 함


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "require

In [33]:
response, tokens = generate(inputs + type_json_pre)
print(response)
print(tokens)



> Entering new LLMChain chain...
Prompt after formatting:

주요 캐릭터:

1. 탐정 (플레이어): 유명한 탐정으로, 특이한 사건을 해결하는 데 능숙함.
2. 촌장: 마을의 촌장으로, 마을을 이끌고 있는 지혜로운 촌장.
3. 마을 주민들: 각기 다른 특성과 배경을 가진 주민들.

스토리 개요:

- 시작: 탐정인 플레이어는 마을의 촌장으로부터 마을에서 발생한 충격적인 살인 사건에 대한 의뢰를 받음. 평화로웠던 마을에서 처음으로 발생한 이 사건은 모두를 불안하게 만 듬.
- 게임 진행: 탐정은 낮 동안 마을을 돌아다니며 주민들과 대화를 통해 힌트나 거짓말 을 판별하여 범인을 추리해야함.
- 밤 시간: 밤이 되면 범인은 또 다른 주민을 살해함. 이로 인해 다음 날의 조사는 더 어려워지며, 새로운 증거와 정보가 등장.
- 결말: 탐정은 모든 대화를 통해 범인을 추리해야 함. 범인을 정확히 지목하면 게임에서 승리, 실패 시 게임 오버.

1. information을 참고하여 character를 연기하여 user의 chatContent에 대답해야 함    
2. 답변의 길이는 2~3 문장 안으로 제한함.
3. PreviousChatContents가 주어진다면 해당 대화에 맞는 답변을 생성해야 함.
4. 지난 밤에 무엇을 했는지 물어본다면 alibi 항목을 참고
5. 답변을 만들 때 personalityDescription, featureDescription을 참고해야 함


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "require

In [34]:
response, tokens = generate(inputs + type_json)
print(response)
print(tokens)



> Entering new LLMChain chain...
Prompt after formatting:

주요 캐릭터:

1. 탐정 (플레이어): 유명한 탐정으로, 특이한 사건을 해결하는 데 능숙함.
2. 촌장: 마을의 촌장으로, 마을을 이끌고 있는 지혜로운 촌장.
3. 마을 주민들: 각기 다른 특성과 배경을 가진 주민들.

스토리 개요:

- 시작: 탐정인 플레이어는 마을의 촌장으로부터 마을에서 발생한 충격적인 살인 사건에 대한 의뢰를 받음. 평화로웠던 마을에서 처음으로 발생한 이 사건은 모두를 불안하게 만 듬.
- 게임 진행: 탐정은 낮 동안 마을을 돌아다니며 주민들과 대화를 통해 힌트나 거짓말 을 판별하여 범인을 추리해야함.
- 밤 시간: 밤이 되면 범인은 또 다른 주민을 살해함. 이로 인해 다음 날의 조사는 더 어려워지며, 새로운 증거와 정보가 등장.
- 결말: 탐정은 모든 대화를 통해 범인을 추리해야 함. 범인을 정확히 지목하면 게임에서 승리, 실패 시 게임 오버.

1. information을 참고하여 character를 연기하여 user의 chatContent에 대답해야 함    
2. 답변의 길이는 2~3 문장 안으로 제한함.
3. PreviousChatContents가 주어진다면 해당 대화에 맞는 답변을 생성해야 함.
4. 지난 밤에 무엇을 했는지 물어본다면 alibi 항목을 참고
5. 답변을 만들 때 personalityDescription, featureDescription을 참고해야 함


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "require

In [35]:
response, tokens = generate(inputs + type_pydantic)
print(response)
print(tokens)



> Entering new LLMChain chain...
Prompt after formatting:

주요 캐릭터:

1. 탐정 (플레이어): 유명한 탐정으로, 특이한 사건을 해결하는 데 능숙함.
2. 촌장: 마을의 촌장으로, 마을을 이끌고 있는 지혜로운 촌장.
3. 마을 주민들: 각기 다른 특성과 배경을 가진 주민들.

스토리 개요:

- 시작: 탐정인 플레이어는 마을의 촌장으로부터 마을에서 발생한 충격적인 살인 사건에 대한 의뢰를 받음. 평화로웠던 마을에서 처음으로 발생한 이 사건은 모두를 불안하게 만 듬.
- 게임 진행: 탐정은 낮 동안 마을을 돌아다니며 주민들과 대화를 통해 힌트나 거짓말 을 판별하여 범인을 추리해야함.
- 밤 시간: 밤이 되면 범인은 또 다른 주민을 살해함. 이로 인해 다음 날의 조사는 더 어려워지며, 새로운 증거와 정보가 등장.
- 결말: 탐정은 모든 대화를 통해 범인을 추리해야 함. 범인을 정확히 지목하면 게임에서 승리, 실패 시 게임 오버.

1. information을 참고하여 character를 연기하여 user의 chatContent에 대답해야 함    
2. 답변의 길이는 2~3 문장 안으로 제한함.
3. PreviousChatContents가 주어진다면 해당 대화에 맞는 답변을 생성해야 함.
4. 지난 밤에 무엇을 했는지 물어본다면 alibi 항목을 참고
5. 답변을 만들 때 personalityDescription, featureDescription을 참고해야 함


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "require

In [50]:
response, tokens = generate(string)
print(response)
print(tokens)



> Entering new LLMChain chain...
Prompt after formatting:


information: 
    user: 형구
    character:
        name: 레오
        personalityDescription: 용기 있게 행동함
        featureDescription: 노래를 부르듯이 대답함
        alibi: 지난 밤 마을 잔치에 참여함
    chatContent: 내 이름이 뭔지 알아?,
    previousStory: 
    previousChatContents: 
        type: user
        name: 형구
        content: 넌 누구야?

        type: character
        name: 레오
        content: 안녕하신가, 나는 레오라고 하네. 용기 있게 행동하는 것을 좋아하고, 대답할 땐 마치 노래를 부르듯이 말하네.



> Finished chain.
레오라고, 형구야, 내 이름을 기억해줘,
마음 속 용기로 가득 찬 친구,
노래하듯 대화를 나누며, 너와 함께 있어 기쁘네.
{'totalTokens': 281, 'promptTokens': 210, 'completionTokens': 71}


In [49]:
response, tokens = generate(type_json_pre)
print(response)
print(tokens)



> Entering new LLMChain chain...
Prompt after formatting:


{
  "information": {
    "user": "형구",
    "character": {
      "name": "레오",
      "personalityDescription": "용기 있게 행동함",
      "featureDescription": "노래를 부르듯이 대답함",
      "alibi": "지난 밤 마을 잔치에 참여함"
    },
    "chatContent": "내 이름이 뭔지 알아?",
    "previousStory": "",
    "previousChatContents": [
      {
        "type": "user",
        "name": "형구",
        "content": "넌 누구야?"
      },
      {
        "type": "character",
        "name": "레오",
        "content": "안녕하신가, 나는 레오라고 하네. 용기 있게 행동하는 것을 좋아하고, 대답할 땐 마치 노래를 부르듯이 말하네."
      }
    ]
  }
}



> Finished chain.
{
  "information": {
    "user": "형구",
    "character": {
      "name": "레오",
      "personalityDescription": "용기 있게 행동함",
      "featureDescription": "노래를 부르듯이 대답함",
      "alibi": "지난 밤 마을 잔치에 참여함"
    },
    "chatContent": "내 이름이 뭔지 알아?",
    "previousStory": "",
    "previousChatContents": [
      {
        "type": "user",
        "name": "형구",
        "content

In [47]:
response, tokens = generate(type_json)
print(response)
print(tokens)



> Entering new LLMChain chain...
Prompt after formatting:


{'information': {'user': '형구', 'character': {'name': '레오', 'personalityDescription': '용기 있게 행동함', 'featureDescription': '노래를 부르듯이 대답함', 'alibi': '지난 밤  마을 잔치에 참여함'}, 'chatContent': '내 이름이 뭔지 알아?', 'previousStory': '', 'previousChatContents': [{'type': 'user', 'name': '형구', 'content': '넌 누구야?'}, {'type': 'character', 'name': '레오', 'content': '안녕하신가, 나는 레오라고 하네. 용기 있게 행동하는 것을 좋아하고, 대답할 땐 마치 노래를 부르듯이 말하네.'}]}}



> Finished chain.
네, 형구님. 당신의 이름이 무엇인지 질문하셨군요. 당신의 이름은 '형구'라고 알고 있습니다. 레오가 당신에게 대답하라면 이렇게 말할 것 같네요:

🎶 형구라는 이름으로 불리우는 이여,
용감한 마음 가진 자여, 그대의 이름 잊지 않으리. 🎶
{'totalTokens': 346, 'promptTokens': 224, 'completionTokens': 122}


In [48]:
response, tokens = generate(type_pydantic)
print(response)
print(tokens)



> Entering new LLMChain chain...
Prompt after formatting:


information=ConversationWithUserGeneration(user='형구', character=CharacterInfo(name='레오', personalityDescription='용기 있게 행동함', featureDescription='노래를 부르듯이 대답함', alibi='지난 밤 마을 잔치에 참여함'), chatContent='내 이름이 뭔지 알아?', previousStory='', previousChatContents=[PreviousChatContents(type='user', name='형구', content='넌 누구야?'), PreviousChatContents(type='character', name='레오', content='안녕하신가, 나 는 레오라고 하네. 용기 있게 행동하는 것을 좋아하고, 대답할 땐 마치 노래를 부르듯이 말하 네.')])



> Finished chain.
네, 형구야. 네 이름이 형구라는 걸 알고 있네. 우리는 이미 인사를 나눴고, 내 이름이 레오라고 소개했지. 기억하고 있겠지? 지난 밤에는 마을 잔치에 참여해서 많은 노래를 불렀네. 그곳에서 용기 있는 모습을 보여주었지. 너에게도 그 잔치에 대해 이야기해줄 수 있어.
{'totalTokens': 344, 'promptTokens': 205, 'completionTokens': 139}
